In [1]:
using Plots
using HDF5
using Unitful
using Statistics
using Distributions
using Plots.PlotMeasures

### Set up the dates array

In [2]:
using Dates

IDate = "1982010100"
endDate = "1982080100"
SPEEDY_DATE_FORMAT = "YYYYmmddHH"
dates = collect(DateTime(IDate, SPEEDY_DATE_FORMAT):Dates.Hour(6):DateTime(endDate, SPEEDY_DATE_FORMAT))

849-element Vector{DateTime}:
 1982-01-01T00:00:00
 1982-01-01T06:00:00
 1982-01-01T12:00:00
 1982-01-01T18:00:00
 1982-01-02T00:00:00
 1982-01-02T06:00:00
 1982-01-02T12:00:00
 1982-01-02T18:00:00
 1982-01-03T00:00:00
 1982-01-03T06:00:00
 ⋮
 1982-07-30T00:00:00
 1982-07-30T06:00:00
 1982-07-30T12:00:00
 1982-07-30T18:00:00
 1982-07-31T00:00:00
 1982-07-31T06:00:00
 1982-07-31T12:00:00
 1982-07-31T18:00:00
 1982-08-01T00:00:00

### Define Functions

In [3]:
function read_hdf5_file(filename, timestamp, field, level)
    fh = h5open(filename, "r")
    # println("The following datasets found in file ", filename, ": ", keys(fh))
    x_st_unit = read(fh["station_coordinates"]["lon"]["Unit"])
    y_st_unit = read(fh["station_coordinates"]["lat"]["Unit"])
    x_st = read(fh["station_coordinates"]["lon"]) .* uparse(x_st_unit) .|> u"°"
    y_st = read(fh["station_coordinates"]["lat"]) .* uparse(y_st_unit) .|> u"°"
    return fh, x_st, y_st
end

function read_hdf5_truth(filename, timestamp, field, unit_comp, level)
    th = h5open(filename, "r")
    field_unit = "hPa"
    field_desc = read(th["state"][timestamp][field]["Description"]);
    z_truth = read(th["state"][timestamp][field]) .* uparse(field_unit);
    return z_truth[:,:,level].*unit_comp
end

function find_gauge_location(gauge_id, gauges_lon, gauges_lat, state_lon, state_lat)
    pointx = findmin((abs.(state_lon .- ustrip(gauges_lon[gauge_id]))))[2]
    pointy = findmin((abs.(state_lat .- ustrip(gauges_lat[gauge_id]))))[2]
    return pointx, pointy
end

function loop_over_particles(fh, timestamp, field, unit_comp, pointx, pointy)
    field_unit = read(fh["state_particle_1"][timestamp][field]["Unit"])
    field_desc = read(fh["state_particle_1"][timestamp][field]["Description"])
    psA = Float64[]
    for it = 1:256
        particle = read(fh[string("state_particle_",string(it))][timestamp][field]);
        push!(psA, particle[pointx,pointy,level]*unit_comp)
    end
    return psA
end

function plotting(particles, z_truth, gauge_id, pointx, pointy, state_lon, state_lat, timestamp, date, output_folder)
    distA = Normal(mean(particles),std(particles))
    targetx = z_truth[pointx,pointy]
    skew, kurt = round(skewness(particles), digits = 3), round(kurtosis(particles), digits = 3)
    longitude = round(state_lon[pointx], digits = 2)
    latitude = round(state_lat[pointy], digits = 2)
    histogram(particles, xlabel="Surface Pressure", label="", linecolor = "white", normalize = true, top_margin = 12px, title="Observation Location ($longitude °, $latitude °) \n $date")
    # histogram(particles, xlabel="Surface Pressure", label="", linecolor = "white", normalize = true, top_margin = 12px, title="Histogram of Surface Pressure at ($longitude °, $latitude °)")# \n Skewness = $skew  Kurtosis = $kurt")
    vline!([targetx], seriestype = :scatter, color=:red, label="True Value")
    plot!(x->pdf(distA, x), label="Fitted Gaussian", bottom_margin = 12px)
    plot!(size=(1000,400), dpi=300)
    vline!([mean(particles)], seriestype = :scatter, color=:black, label="Mean Value")

    sub_dir = joinpath(output_folder,timestamp) 

    if isdir(sub_dir) == false
        mkdir(sub_dir)
    end
    
    savefig(joinpath(sub_dir,string(gauge_id)))
    
    # if abs(skew) > 0.5 || abs(kurt) > 1.0
    #     @show gauge_id, timestamp
    # end
end

plotting (generic function with 1 method)

# Load Filter output file

In [4]:
timestamp = ["t0000", "t0025", "t0050", "t0075", "t0100", "t0125", "t0150", "t0175", "t0200", "t0225", "t0250"]
field = "ps" # Choose from the fields listed above
unit_comp = 0.01 # Convert Pa to hPa
field_unit = "hPa"
output_folder = ""
level = 1

truth_filename = joinpath(output_folder, "obs.h5")
filename = joinpath(output_folder, "optimal.h5")
plot_folder = joinpath(output_folder, "particles")
#Loop through all the gauges and produce the distributions
fh, x_st, y_st = read_hdf5_file(filename, timestamp[1], field, level)
gauges_lon = ustrip(x_st)
gauges_lat = ustrip(y_st)
state_lon = range(0,180,48)
state_lat = range(0,360,96)
for time in timestamp
    number=parse(Int64,split(time,'t')[2]);
    @show time, dates[number+1]
    fh, x_st, y_st = read_hdf5_file(filename, time, field, level)
    z_truth = read_hdf5_truth(truth_filename, time, field, unit_comp, level)
    for gauge_id = 1:50
        pointx, pointy = find_gauge_location(gauge_id, gauges_lon, gauges_lat, state_lon, state_lat)
        particles = loop_over_particles(fh, time, field, unit_comp, pointx, pointy)
        plotting(particles, z_truth, gauge_id, pointx, pointy, state_lon, state_lat, time, dates[number+1], plot_folder)
    end
end

ErrorException: unable to determine if optimal.h5 is accessible in the HDF5 format (file may not exist)

In [12]:
output_folder = "/Users/dangiles/Documents/Stats/RADDISH/ParticleDA-UseCases/SPEEDY/results"
truth_filename = joinpath(output_folder, "uniform_obs_Jan_Aug.h5")
for time in timestamp
    z_truth = read_hdf5_truth(truth_filename, time, field, unit_comp, level)
    @show z_truth.size
end

z_truth = Quantity{Float64, 𝐌 𝐋⁻¹ 𝐓⁻², Unitful.FreeUnits{(hPa,), 𝐌 𝐋⁻¹ 𝐓⁻², nothing}}[727.0237759305827 hPa 768.5814667312956 hPa 730.0591001073716 hPa 726.026235509477 hPa 851.4366585755346 hPa 997.3807700732042 hPa 1031.2679235740443 hPa 1020.9503945926344 hPa 1024.306169401004 hPa 1021.5457354228205 hPa 1023.6563705689192 hPa 1032.664196535889 hPa 1027.7927119076148 hPa 1026.4800970658748 hPa 1036.809003673399 hPa 1030.9527767171508 hPa 1016.375660685246 hPa 1014.9688493747431 hPa 1012.1884207450987 hPa 1006.7896291086747 hPa 1014.5404477363966 hPa 1018.1252422620256 hPa 1008.8285786555797 hPa 1011.3692930677953 hPa 1018.5663538008201 hPa 1004.538817044314 hPa 986.3641754436165 hPa 983.1490290550031 hPa 982.9351257082541 hPa 982.3870456304114 hPa 982.668411102363 hPa 969.9148725486324 hPa 952.2620269851072 hPa 955.3480658763563 hPa 973.8973272677182 hPa 986.5531960445385 hPa 994.8089461770236 hPa 1007.2794577417284 hPa 1016.5840709384252 hPa 1007.1290345326863 hPa 977.8090511845344 

Excessive output truncated after 584526 bytes.